In [ ]:
library(data.table)
library(rvest)
library(magrittr)
library(stringr)

In [ ]:
url <- "https://www.weltfussball.de/alle_spiele/bundesliga-2018-2019/"

content <- read_html(url)

In [ ]:
Spieldatum <- content %>%
  html_nodes(".portfolio .standard_tabelle") %>%
  html_nodes("td:nth-of-type(1)") %>% # get first column of table
  html_text()

Spieldatum %>% head()

In [ ]:
Heimteam <- content %>%
  html_nodes(".portfolio .standard_tabelle") %>%
  html_nodes("td:nth-of-type(3)") %>% # get third column of table
  html_text()

Heimteam %>% head()

In [ ]:
Heimteam <- Heimteam %>% str_replace_all(" ", "") 

Heimteam %>% head()

In [ ]:
Austeam <- content %>%
  html_nodes(".portfolio .standard_tabelle") %>%
  html_nodes("td:nth-of-type(5)") %>% # get fifth column of table
  html_text()

Austeam %>% head()

In [ ]:
Austeam <- Austeam %>% str_replace_all(" ", "")

Austeam %>% head()

In [ ]:
Ergebnis <- content %>%
  html_nodes(".portfolio .standard_tabelle") %>%
  html_nodes("td:nth-of-type(6)") %>% # get sixth column of table
  html_node("a") %>% 
  html_text()

Ergebnis %>% head()

In [ ]:
Ergebnis <- Ergebnis %>% str_extract("^\\d+:\\d+")

Ergebnis %>% head()

In [ ]:
resultTable <- data.table(Spieldatum,
                          Heimteam, 
                          Austeam,
                          Ergebnis)

resultTable %>% head()

In [ ]:
# fill the missing dates with the previous ones.
resultTable[, Spieldatum := Spieldatum[1], .(cumsum(Spieldatum != ""))]

resultTable %>% head()

In [ ]:
# split the date into 3 columns
resultTable[, c("Tag", "Monat", "Jahr") := tstrsplit(Spieldatum, ".", fixed = TRUE)]

resultTable %>% head()

In [ ]:
# split the result into 2 columns
resultTable[, c("Heim", "Aus") := tstrsplit(Ergebnis, ":", fixed = TRUE)]

resultTable %>% head()

In [ ]:
# add the dummy column "Bericht" to make the data readable in Excel
resultTable[, Bericht := "Bericht"]

resultTable %>% head()